## 简洁实现循环神经网络

In [4]:
import time
import math
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F
import zipfile

import sys
sys.path.append("..") 
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def load_data_jay_lyrics():
    """加载歌词数据集"""
    with zipfile.ZipFile('../data/jaychou/jaychou_lyrics.txt.zip') as zin:
        with zin.open('jaychou_lyrics.txt') as f:
            corpus_chars = f.read().decode('utf-8')                             # corpus_chars 以字符形式读入数据
    corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')           # 将其中的换行符换为空格
    corpus_chars = corpus_chars[0:10000]                                        # 取前10000个字符作为训练的数据
    idx_to_char = list(set(corpus_chars))                                       # idx_to_char 按照字典顺序排列的汉字列表 如：{'去', '化', ' ', '和', '要',}
    char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])       # char_to_idx "汉字": "对应的索引" 形式的字典 如：{'去': 0, '化': 1, ' ': 2, '和': 3, '要': 4, '这': 5}
    vocab_size = len(char_to_idx)                                               # vocab_size 词典中不同字符的个数
    corpus_indices = [char_to_idx[char] for char in corpus_chars]               # corpus_indices 歌词顺序的每个字符对应字典中的索引
    return corpus_indices, char_to_idx, idx_to_char, vocab_size

(corpus_indices, char_to_idx, idx_to_char, vocab_size) = load_data_jay_lyrics()


### 定义模型
构造单隐藏层、隐藏单元

In [5]:
num_hiddens = 256
# rnn_layer = nn.LSTM(input_size=vocab_size, hidden_size=num_hiddens) # 已测试
rnn_layer = nn.RNN(input_size=vocab_size, hidden_size=num_hiddens)

In [10]:
num_steps = 35
batch_size = 2
state = None
X = torch.rand(num_steps, batch_size, vocab_size)
Y, state_new = rnn_layer(X, state)
print(X.shape, Y.shape, len(state_new), state_new[0].shape)

torch.Size([35, 2, 1027]) torch.Size([35, 2, 256]) 1 torch.Size([2, 256])


In [11]:
# 本类已保存在d2lzh_pytorch包中方便以后使用
class RNNModel(nn.Module):
    def __init__(self, rnn_layer, vocab_size):
        super(RNNModel, self).__init__()
        self.rnn = rnn_layer
        self.hidden_size = rnn_layer.hidden_size * (2 if rnn_layer.bidirectional else 1) 
        self.vocab_size = vocab_size
        self.dense = nn.Linear(self.hidden_size, vocab_size)
        self.state = None

    def forward(self, inputs, state): # inputs: (batch, seq_len)
        # 获取one-hot向量表示
        X = d2l.to_onehot(inputs, self.vocab_size) # X是个list
        Y, self.state = self.rnn(torch.stack(X), state)
        # 全连接层会首先将Y的形状变成(num_steps * batch_size, num_hiddens)，它的输出
        # 形状为(num_steps * batch_size, vocab_size)
        output = self.dense(Y.view(-1, Y.shape[-1]))
        return output, self.state

In [12]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def predict_rnn_pytorch(prefix, num_chars, model, vocab_size, device, idx_to_char,
                      char_to_idx):
    state = None
    output = [char_to_idx[prefix[0]]] # output会记录prefix加上输出
    for t in range(num_chars + len(prefix) - 1):
        X = torch.tensor([output[-1]], device=device).view(1, 1)
        if state is not None:
            if isinstance(state, tuple): # LSTM, state:(h, c)  
                state = (state[0].to(device), state[1].to(device))
            else:   
                state = state.to(device)

        (Y, state) = model(X, state)
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(int(Y.argmax(dim=1).item()))
    return ''.join([idx_to_char[i] for i in output])


In [13]:
model = RNNModel(rnn_layer, vocab_size).to(device)
predict_rnn_pytorch('分开', 10, model, vocab_size, device, idx_to_char, char_to_idx)

'分开通肉B故啸啸尽尽带乐'

In [14]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes):
    loss = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    model.to(device)
    state = None
    for epoch in range(num_epochs):
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = d2l.data_iter_consecutive(corpus_indices, batch_size, num_steps, device) # 相邻采样
        for X, Y in data_iter:
            if state is not None:
                # 使用detach函数从计算图分离隐藏状态, 这是为了
                # 使模型参数的梯度计算只依赖一次迭代读取的小批量序列(防止梯度计算开销太大)
                if isinstance (state, tuple): # LSTM, state:(h, c)  
                    state = (state[0].detach(), state[1].detach())
                else:   
                    state = state.detach()

            (output, state) = model(X, state) # output: 形状为(num_steps * batch_size, vocab_size)

            # Y的形状是(batch_size, num_steps)，转置后再变成长度为
            # batch * num_steps 的向量，这样跟输出的行一一对应
            y = torch.transpose(Y, 0, 1).contiguous().view(-1)
            l = loss(output, y.long())

            optimizer.zero_grad()
            l.backward()
            # 梯度裁剪
            d2l.grad_clipping(model.parameters(), clipping_theta, device)
            optimizer.step()
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]

        try:
            perplexity = math.exp(l_sum / n)
        except OverflowError:
            perplexity = float('inf')
        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, perplexity, time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn_pytorch(
                    prefix, pred_len, model, vocab_size, device, idx_to_char,
                    char_to_idx))


In [16]:
num_epochs, batch_size, lr, clipping_theta = 250, 32, 1e-3, 1e-2 # 注意这里的学习率设置
pred_period, pred_len, prefixes = 50, 50, ['只剩下钢琴陪我', '不分开']
train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                            corpus_indices, idx_to_char, char_to_idx,
                            num_epochs, num_steps, lr, clipping_theta,
                            batch_size, pred_period, pred_len, prefixes)

epoch 50, perplexity 1.012056, time 0.06 sec
 - 只剩下钢琴陪我 一定是我们在痛痛 我杰伦   我没有你说了 我想大声宣布 对你依依不舍 连隔壁邻居都猜到我现在的感
 - 不分开不了我不要再想 我不知我觉 我不再这样 我不要再这样的不从 让她人在黑离还我开始 家 太沉  分怕眼
epoch 100, perplexity 1.040762, time 0.06 sec
 - 只剩下钢琴陪我去就这样的完快就像 一壶 酒 再来一碗热粥 配上几斤的牛肉 我说店小二 三两银够不够 景色入秋 漫天
 - 不分开不了我不要再想你 爱你看棒  就这样没担忧 唱着歌 一直走 我想就这样牵着你的手不放开 爱可不可以简
epoch 150, perplexity 1.008701, time 0.07 sec
 - 只剩下钢琴陪我 就这样的生活 我爱你 你叫我妈妈妈 这样什么小到 怎么每天祈祷我的心跳你知道  杵在伊斯坦堡 却只
 - 不分开不了我不要再想你 爱情来的太快就像龙卷风 离不开暴风圈来不及逃 我不能再想 我不能再想 我不 我不 
epoch 200, perplexity 1.008469, time 0.06 sec
 - 只剩下钢琴陪我 在这样对着手 一阵莫 感动 我想带你 回我的外婆家 一起看着日落 一直到我们都睡着 我想就这样牵着
 - 不分开 我打我妈  我说你爸 你打我妈 这样对吗干嘛这样 何必让酒牵鼻子走 瞎 说都说不听听 痛是我们在痛
epoch 250, perplexity 1.008043, time 0.07 sec
 - 只剩下钢琴陪我 就这样的节奏 谁都无 奈何 没有你看棒  不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉
 - 不分开不了我不要再想  不知不觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 
